## Preliminaries

### Installing and starting Interactive Brokers TWS 

Visit the <a href='https://www.interactivebrokers.com/en/home.php'>Interactive Brokers</a> website and click on the button 
**Free Trial**.

<img src="http://web-training2.pqp.io/training2/schwed/start_ib.png" width="80%" align="left"><br>

Follow the installation instructions from Interactive Brokers.

When the installation finished Interactive Broker's Traders Workstation (TWS) starts automatically. If not, click on the TWS icon.

First, a login pop-up appears.

<img src="http://web-training2.pqp.io/training2/schwed/login_ib.png" width="50%" align="left"><br>

Login as user **edemo** and password **demouser** and select **Paper Trading**.
### Caution
Do **not** use the login credentials you used when you register for the Interactive Trial.

### Configure TWS for API Usage

In the main menu of the TWS screen, click  `File --> GlobalConfiguration`.

<img src="http://web-training2.pqp.io/training2/schwed/TWS_conf.png" width="50%" align="left"><br>

The configuration window opens. There, click on `API` in the menu on the left hand side. 

<img src="http://web-training2.pqp.io/training2/schwed/TWS_settings.png" width="50%" align="left"><br>

If you then click on the item `Settings`, a new form appears. Select or unselect the items as in the screenshot below. 

<img src="http://web-training2.pqp.io/training2/schwed/TWS_api.png" width="50%" align="left"><br>

Then click on the item `Precaution`. On the new form select the first checkbox.

<img src="http://web-training2.pqp.io/training2/schwed/TWS_api2.png" width="50%" align="left"><br>

Now, your TWS is ready to accept incoming connections from the Python API.

## Installing ibpy

tpqib is based on ibpy, a Python wrapper library for the TWS API. 
The code and detailed installing instructions can be found <a href='https://github.com/blampe/IbPy'>here</a>.

Basically, you need to do the following:

`git clone https://github.com/blampe/IbPy.git`

`cd IbPy`

`python setup.py install`

## Using tpqib

After installing TWS and ibpy, you are ready to test the API. Please ensure that your local TWS is running. 

First, we import our class.

In [1]:
import tpqib
import datetime as dt
import seaborn as sns; sns.set()
%matplotlib inline

Making a new connection is easy:

In [2]:
conn = tpqib.tpqibcon()

Server Version: 76
TWS Time at connection:20161010 11:27:32 CET


Your local TWS will ask for a confirmation of the connection, if you want to proceed click `yes`.

<img src="http://web-training2.pqp.io/training2/schwed/TWS_confirm.png" width="50%" align="left"><br>

You can avoid being prompted for confirmation by adding `127.0.0.1` to the field `Trusted IPs` in the API configuration of the TWS application.

### The contract object

After a connection is established, we can define contract objects. The syntax is

**create_contract(*symbol*, *security type*, *primary exchange*, *exchange*, *currency*)**

For detailed information about the parameters see <a href='http://interactivebrokers.github.io/tws-api/basic_contracts.html#gsc.tab=0'>here</a>. For the moment, we will define a contract for Google stock in USD, we let Interactive Broker choose the exchange by their *SMART* algorithm.


In [ ]:
goog_contract = conn.create_contract('GOOG', 'STK', 'SMART', 'SMART', 'USD')

The same for Apple stock.

In [ ]:
aapl_contract = conn.create_contract('AAPL', 'STK', 'SMART', 'SMART', 'USD')

You can examine the contract object:

In [ ]:
conn.req_contract_details(aapl_contract)

Or get market data for it:

In [ ]:
conn.get_market_data_once(aapl_contract)

Please note that the **data requests are asynchronous**. That means that the notebook cell above will appear as finished before the data is received.

Also, the demo version of the TWS is quite slow, so please be patient.

## Defining and placing orders

Before placing an order, we need to define an order object. This is done by the method

**create_order(*order type*, *quantity*, *action*)**

More on available order types can be found on the <a href='http://interactivebrokers.github.io/tws-api/basic_orders.html#gsc.tab=0'> TWS API sites</a>.

We define an order to buy 100 shares of whatever on market conditions.

In [ ]:
buy_100_order = conn.create_order('MKT', 100, 'Buy')

The following executes such an order for the Apple contact.

In [ ]:
conn.place_order(aapl_contract, buy_100_order)

**Congratulations**, you just bought 100 shares of Apple!

A small note about order ids. 
If you are already familiar with one of the Interactive Brokers APIs (either their own ones in Java or C++ or ibpy), you know that every order has its own unique order id (an integer which is increased by one with every order) and that you must take care of that id for your own. 
An tpqib object handles that job for you. When initializing the object, tpqib synchronizes the order id with your TWS application and counts it up for every transaction.

So, lets look for the next order id, buy another 100 shares of Apple and examine the order id again.

In [ ]:
conn.get_next_order_id()

In [ ]:
conn.place_order(aapl_contract, buy_100_order)

In [ ]:
conn.get_next_order_id()

### The portfolio

Lets have a look at our portfolio:

In [ ]:
conn.req_positions()

### Loading historical data

You can load historical data with the method

**request_historical_data(*contract*, *end_date_time*, *duration*, *bar_size*, *what_to_show*, *use_RTH*, *format_date*)**

A detailed description of the parameters can be found on the <a href='http://interactivebrokers.github.io/tws-api/historical_data.html#hd_receive&gsc.tab=0'> TWS API sites</a>.

As the `request_market_data` method, `request_historical_data` returns a request id. That id is used to get the data from the  tpqib object.

In the example below, we load the historical prices of Apple from today and up to 5 days in the past for 2 hours bars.

In [ ]:
end_date = dt.datetime.now()
request_id = conn.request_historical_data(aapl_contract, end_date,
                                          '1 W', '8 hours', 'TRADES')

In [ ]:
conn.get_historical_data(request_id)

When the data is still loading due to a call of ***get_historical_data(request_id)***, the method returns *False*. 

In [ ]:
request_id = conn.request_historical_data(aapl_contract, end_date,
                                        '1 W', '2 hours', 'TRADES')
conn.get_historical_data(request_id)

If you want to suppress the 'still loading' message, call the method with the argument ***silent=True***.

In [ ]:
request_id = conn.request_historical_data(aapl_contract, end_date,
                                          '1 W', '2 hours', 'TRADES')
data = conn.get_historical_data(request_id, silent=True)
print(data)

To check, if a historical data request is still loading, you can also use the method ***is_historical_data_loading(request_id)***.

The method returns `True` if the request has not yet finished loading and `False` else.

In [ ]:
request_id = conn.request_historical_data(aapl_contract, end_date,
                                          '1 W', '1 min', 'TRADES')
print('loading the data')
while conn.is_historical_data_loading(request_id):
    pass    
print('finished loading')

Now data is ready, let's use it to perform a backtest of a simple moving average strategy.

In [ ]:
data = conn.get_historical_data(request_id)

In [ ]:
data.head()

The short leg should be the average of the last ten minutes, the long leg the average of the last 60 minutes.
The first step is to resample the pandas DataFrame object to 10 minutes timesteps and add the spot closing price to the new dataset:

In [ ]:
resampled_data = data.resample('10min').mean()
resampled_data['spot_close'] = data['close']

Lets compare the two datasets.

In [ ]:
resampled_data[['spot_close', 'close']].plot(figsize=(10, 6));

Next, we define the long leg, which is the rolling mean of the last 6 close prices.

In [ ]:
resampled_data['long'] = resampled_data['close'].rolling(6).mean()
resampled_data[['close', 'long']].plot(figsize=(10, 6));

To find out if the short leg (which is equal to the close price of the resampled dataset) crosses the long leg, we first compare the two legs:

In [ ]:
resampled_data['state'] = resampled_data.apply(
                        lambda x: -1 if (x['close'] < x['long'])
                        else 1 if x['long'] > 1 else 0, axis=1 )

In [ ]:
resampled_data.head(20)

A crossing of the two lines appears if the sign of two consecutive states changes.

In [ ]:
resampled_data['help'] = (resampled_data['state'] +
                          resampled_data['state'].shift(1))
resampled_data['execute'] = resampled_data.apply(
                            lambda x: 1 if x['state'] != 0
                            and x['help'] == 0 else None, axis=1)

In [ ]:
resampled_data.head(20)

In [ ]:
resampled_data[['close', 'long']].plot(figsize=(10, 6));
(resampled_data['spot_close'] * resampled_data['execute']).plot(style='r.');

### Streaming market data 

You can stream market data for a contract by using the method ***request_market_data(contract, callback)***.

Every data tick received from TWS is sent to the function given as the callback parameter during the method call.

That function should take two arguments, `field` and `value`, where `field` is the type of data received and `value` is its value.

`field` can be from the following list:

In [ ]:
field = ['lastTimestamp', 'askPrice', 'askSize',
         'bidPrice', 'bidSize',
         'low', 'high', 'close',
         'volume', 'lastPrice', 'lastSize', 'halted']

The method returns a request id. That id is used to stop streaming. 

**Example:** The following function prints all streamed bid prices for Google

In [ ]:
def print_bid_price(field, value):
    if field == 'bidPrice':
        print('bid for %s' % value)

In [ ]:
request_id = conn.request_market_data(goog_contract, print_bid_price)

To **stop**, execute the cell below.

In [ ]:
conn.cancel_market_data(request_id)

### An Example with Order Execution

Let us stream market data for Google and buy 10 shares every time the bid price is below 1000 (and it is, at least at the moment) until there is no money left. 

First, we define a new order object.

In [ ]:
buy_10_order = conn.create_order('MKT', 10, 'Buy')

The capital we are willing to invest.

In [ ]:
money = 50000

And the respective callback function.

In [ ]:
def buy_or_bust(field, value):
    global money
    global request_id
    if field == 'bidPrice' and value < 1000:
        print('10 shares cost %s USD, I have %s USD, so .. '
              % (10 * value, money))
        if 10 * value < money:
            print('I buy!')
            conn.place_order(goog_contract, buy_10_order)
            money = money - 10 * value
        else:
            print('I quit')
            conn.cancel_market_data(request_id)

Let us start the algorithm.

In [ ]:
request_id = conn.request_market_data(goog_contract, buy_or_bust)

A view on the resulting portfolio positions.

In [ ]:
conn.req_positions()

Finally, it is time to close the connection.

In [ ]:
conn.close()